# Test random varible evaluation

For first step we create import the sufficient packages 

In [1]:
import sys
import os
import time
sys.path.append('../')
from holoclean import holocleansession , dataset , dataengine
from holoclean.utils import domainpruning
from holoclean.featurization import dcfeaturizer

In [2]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../holoclean/mysql-connector-java-5.1.44-bin.jar")
holoclean_se=holocleansession.HolocleanSession(path)

##  Creating Dataset and dataengine and holocleansession 
In this part we create dataset for keep tracking the runnig step and dataengine for make connector to the database we try to ingest the 10 element csv file and after ingesting it returns df other wise we can read it from database

In [3]:
ds=dataset.Dataset()  
d=dataengine.Dataengine("metadb-config.txt",'datadb-config.txt',ds,holoclean_se.return_sqlcontext())
holoclean_se.set_dataengine(d)
df=d.ingest_spark('10.csv',holoclean_se.returnspark_session())

index


##  Create featurizer


In [4]:
dcCode=['t1&t2&EQ(t1.city,t2.city)&EQ(t1.temp,t2.temp)&IQ(t1.tempType,t2.tempType)']
dcf=dcfeaturizer.DCFeaturizer(df,dcCode,d)

In [5]:
start_time_spark = time.time()
dgf=dcf.pre_features(holoclean_se.returnspark_session())
finish_time_spark=time.time()
dgf.show(dgf.count())

/home/alireza/anaconda2/lib/python2.7/site-packages/pyspark/sql/session.py:331: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-------------+-----------+--------------------+------+
|           rv|   assigned|                  dc|tup_id|
+-------------+-----------+--------------------+------+
|     t_1.temp|       -125|t1&t2&EQ(t1.city,...|     7|
|     t_1.temp|       -148|t1&t2&EQ(t1.city,...|     2|
|     t_2.temp|        -60|t1&t2&EQ(t1.city,...|     6|
|     t_2.temp|        -75|t1&t2&EQ(t1.city,...|     1|
| t_3.tempType|       TMAX|t1&t2&EQ(t1.city,...|     8|
| t_3.tempType|       TMIN|t1&t2&EQ(t1.city,...|     8|
|     t_4.temp|       -135|t1&t2&EQ(t1.city,...|    10|
|     t_5.city|EZE00100082|t1&t2&EQ(t1.city,...|    10|
| t_5.tempType|       TMAX|t1&t2&EQ(t1.city,...|    10|
| t_5.tempType|       PRCP|t1&t2&EQ(t1.city,...|    10|
|     t_5.temp|       -135|t1&t2&EQ(t1.city,...|    10|
|     t_6.temp|       -125|t1&t2&EQ(t1.city,...|     7|
|     t_6.temp|       -148|t1&t2&EQ(t1.city,...|     2|
|     t_7.temp|        -60|t1&t2&EQ(t1.city,...|     6|
|     t_7.temp|        -75|t1&t2&EQ(t1.city,...|

Execution time is

In [7]:
print("--- %s seconds ---" % (finish_time_spark-start_time_spark))

--- 535.392323017 seconds ---


By using mysql we have same table :

In [6]:
start_time_mysql = time.time()
dcf.make_dc_f_table()
finish_time_mysql=time.time()
mysqlv=d.get_table_spark("dc_f")
print(mysqlv.show(mysqlv.count()))

438288925569_T
OK!
+--------+--------+------------+--------------------+------+
|rv_index| rv_attr|assigned_val|              DCname|tup_id|
+--------+--------+------------+--------------------+------+
|      10|    city| EZE00100082|table1.city=table...|     5|
|       5|    city| EZE00100082|table1.city=table...|    10|
|      10|    date|    18000101|table1.city=table...|     5|
|      10|    date|    18000102|table1.city=table...|     5|
|       5|    date|    18000101|table1.city=table...|    10|
|       5|    date|    18000102|table1.city=table...|    10|
|      10|tempType|        PRCP|table1.city=table...|     5|
|      10|tempType|        TMIN|table1.city=table...|     5|
|       3|tempType|        TMAX|table1.city=table...|     8|
|       3|tempType|        TMIN|table1.city=table...|     8|
|       5|tempType|        PRCP|table1.city=table...|    10|
|       5|tempType|        TMAX|table1.city=table...|    10|
|       8|tempType|        TMAX|table1.city=table...|     3|
|    

Execution time is :

In [8]:
print("--- %s seconds ---" % (finish_time_mysql-start_time_mysql))

--- 0.268074989319 seconds ---


Because we have just one dc in all value in dc column is same